<a href="https://www.kaggle.com/code/esantosdata/titanic-dataset-applying-ml-model-on-datas-pt2?scriptVersionId=222906267" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importando novamente as bases e fazendo o tratamento de dados

* Vamos apenas replicar o que fizemos no "Primeiro arquivo dessa análise" <a href="https://www.kaggle.com/code/esantosdata/titanic-dataset-applying-ml-model-on-datas" target="_blank">(para visualizar a parte 1 visite este link)</a>

In [1]:

import numpy as np  # Algebra linear
import pandas as pd # Processamento de dados
import os           #Kaggle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# Visualizando a base de treino
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Visualizando a base de teste
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#  **Fazendo o mesmo tratamento inicial que fizemos anteriormente**

In [4]:
# Eliminando as colunas com alta cardinalidade
train_data = train_data.drop(['Name', 'Ticket', 'Cabin'],axis=1)
test_data = test_data.drop(['Name', 'Ticket', 'Cabin'],axis=1)


In [5]:
# Usando a média para substituir os valores nulos na coluna idade
train_data.loc[train_data.Age.isnull(),'Age'] = train_data.Age.mean()
test_data.loc[test_data.Age.isnull(),'Age'] = test_data.Age.mean()

In [6]:
# Tratando a coluna Embarked da base treino usando moda
train_data.loc[train_data.Embarked.isnull(),'Embarked'] = train_data.Embarked.mode()[0]

In [7]:
# Tratando a coluna Fare da base de teste usando a média
test_data.loc[test_data.Fare.isnull(),'Fare'] = test_data.Fare.mean()

# **Podemos agora tratar as colunas de texto**

* Vamos agora **adicionar um novo tratamento nas colunas de texto.**

In [8]:
#Verificando as colunas de texto na base train_data
train_data.columns[train_data.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [9]:
# Verificando os valores da coluna "Sex"
train_data.Sex.value_counts()


Sex
male      577
female    314
Name: count, dtype: int64

In [10]:
# Verificando os valores da coluna "Embarked"
train_data.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

* Para tratar a coluna **"Sex"**, podemos criar uma nova coluna chamada **"Male_Check"** que vai receber **1** se o gênero for **masculino** e **0** se o gênero for **feminino**

In [11]:
# Usando lambda function para fazer este tratamento
# Basicamente estamos criando uma nova coluna na tabela que os valores são resultantes da lambda function que verifica na coluna Sex, se x for igual a male ele será 1 caso contrário será 0
train_data['Male_Check'] = train_data.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [12]:
# Verificando valores
train_data[['Sex','Male_Check']].value_counts()


Sex     Male_Check
male    1             577
female  0             314
Name: count, dtype: int64

#  **Vamos fazer o mesmo para a base test_data**

In [13]:
test_data['Male_Check'] = test_data.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [14]:
test_data[['Sex','Male_Check']].value_counts()

Sex     Male_Check
male    1             266
female  0             152
Name: count, dtype: int64

 * Agora podemos tratar a coluna "Embarked" usando o [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) que ira criar uma nova coluna para cada um dos rotulos da coluna original

In [15]:
# Importando o OneHitEncoder
from sklearn.preprocessing import OneHotEncoder

In [16]:
# Criando o encoder 
#dtype esta no final pois por padrao ele entrega em float
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [17]:
# Fazendo fit dos dados
ohe = ohe.fit(train_data[['Embarked']])

In [18]:
# Fazendo a transformaçao dos dados
ohe.transform(train_data[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=int32)

In [19]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(train_data[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(5)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [20]:
# Agora vamos adicionar essa coluna na train_base
train_data = pd.concat([train_data,ohe_df],axis=1)


In [21]:
# Verificando os valores
train_data[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
Name: count, dtype: int64

# **Agora vamos fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

* E valido comentar que nesta etapa **nao fazemos o fit nos dados pois fizemos com a train_data e depois transformamos na base test_data.**

In [22]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(test_data[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(5)

,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1


In [23]:
# Adicionando o resultado na test_data
test_data = pd.concat([test_data,ohe_df],axis=1)

In [24]:
# Verificando os valores
test_data[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

# **Usando essa nova base no modelo**

In [25]:
# Visualizando a base
train_data.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Male_Check,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [26]:
# Apagando as colunas ja tratadas em ambas as bases
train_data = train_data.drop(['Sex', 'Embarked'], axis=1)
test_data = test_data.drop(['Sex', 'Embarked'], axis=1)

     * Utilizando estes metodos de limpeza de dados na base, conseguimos ter este resultado, que ira nos ajudar na acuracia do desafio.

* **Podemos selecionar os mesmos modelos que utilizamos anteriormente <a href="https://www.kaggle.com/code/esantosdata/titanic-dataset-applying-ml-model-on-datas" target="_blank">(PARTE 1)</a>**
1. **Arvore de classificacao** https://scikit-learn.org/stable/modules/tree.html#classification
2. **Classificacao dos vizinhos mais proximos** https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
3. **Regressao Logistica** https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression


* Antes de usarmos estes algoritmos, precisamos separar a base_train em train_val

    Vamos fazer isso utilizando o train_test_split https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [27]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [28]:
# Separando a base_train em x e y
X = train_data.drop(['PassengerId','Survived'],axis=1)
y = train_data.Survived

In [29]:
# Separando em treino e validaçao
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

# **UTILIZAÇAO DOS MODELOS**

* Para a **Arvore de Classificaçao**

In [30]:
# Fazendo a importaçao
from sklearn import tree

In [31]:
# Criando classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [32]:
# Fazendo fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

In [33]:
# Fazendo a previsao
y_pred_ac = clf_ac.predict(X_val)

* Para o **KNeighborsClassifier**

In [34]:
# Fazendo a importaçao
from sklearn.neighbors import KNeighborsClassifier

In [35]:
# Criando classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [36]:
# Fazendo fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

In [37]:
# Fazendo a previsao
y_pred_knn = clf_knn.predict(X_val)

* E para a **Regressao logistica**

In [38]:
# Fazendo a importaçao
from sklearn.linear_model import LogisticRegression

In [39]:
# Criando classificador
clf_rl = LogisticRegression(random_state=42, max_iter=500)

In [40]:
# Fazendo fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

In [41]:
# Fazendo a previsao
y_pred_rl = clf_rl.predict(X_val)

* Vamos novamente **avaliar os modelos**

* **Acuracia** (metodo de avaliaçao da competiçao): https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
* **Matriz de confusao** (ajuda a visualizar a distribuiçao dos erros): https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


    * Acuracia:

In [42]:
# Importando
from sklearn.metrics import accuracy_score

In [43]:
# Para a Arvore de classificaçao
accuracy_score(y_val, y_pred_ac)

0.7491525423728813

In [44]:
# Para o KNN
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

In [45]:
# Para a Regressao logistica
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

    * Matriz de confusao:

In [46]:
# Importando
from sklearn.metrics import confusion_matrix

In [47]:
# Para a Arvore de classificaçao
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 37,  83]])

In [48]:
# Para o KNN
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]])

In [49]:
# Para a Regressao logistica
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]])

# Fazendo a previsao para os dados de teste

In [50]:
# Visualizando o X_train
X_train.head(5)

,Pclass,Age,SibSp,Parch,Fare,Male_Check,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0,0,1
718,3,29.699118,0,0,15.5000,1,0,1,0
685,2,25.000000,1,2,41.5792,1,1,0,0
73,3,26.000000,1,0,14.4542,1,1,0,0
882,3,22.000000,0,0,10.5167,0,0,0,1


In [51]:
## Visualizando a base de teste (apenas com valores numericos)
train_data.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Male_Check,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,0,1


In [52]:
# Para a base de teste ser igual a base de treino, retiramos a coluna 'PassengerId'
X_test_data = test_data.drop("PassengerId",axis=1)

In [53]:
# Utilizando a regressao logistica na base teste
y_pred = clf_rl.predict(X_test_data)

In [54]:
# Criando uma nova coluna na previsao na base de teste
test_data['Survived'] = y_pred

In [55]:
# Selecionando apenas a colunia PassengerId e Survived para fazer o envio
base_envio = test_data[['PassengerId','Survived']]

In [56]:
# Exportando para um CSV
base_envio.to_csv('resultados3.csv',index=False)

![](https://i.imgur.com/5Afr0PU.png)

* Como podemos ver ao fazer uma analise mais detalhada, tratando e levando em considaraçao as colunas de texto e convertendo as mesmas para nossos modelos, **obtivemos cerca de 10% a mais de acuracia em relaçao ao <a href="https://www.kaggle.com/code/esantosdata/titanic-dataset-applying-ml-model-on-datas" target="_blank">ultimo modelo</a>.**